<a href="https://colab.research.google.com/github/Mahrokh-Eb/TensorflowLearning/blob/main/TF_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start Tensorflow 

In [2]:
# pip install tensorflow